In [1]:
import unittest
from collections import defaultdict
import pdb
import datetime
from decimal import *
getcontext().prec = 6

In [2]:
init_price_token = [datetime.datetime(1971,1,1,0,0,0).timestamp(), 0]

In [3]:
class Fund:
    # ToDo - Replace token_name -> token_address

    price_refresh_deadline = 900 # 15min
    
    def __init__(self):
        self.token_balances = {'WETH': 0, 'DAI': 0}
        self.token_prices = {'WETH': init_price_token, 'DAI': init_price_token}
        self.shareholders_share = {}
        
    def is_price_too_old(self, old_price_retrieval_timestamp):
        return (self.now() - old_price_retrieval_timestamp) > self.price_refresh_deadline
    
    def get_price(self, token_name):
        if token_name == 'WETH':
            return Decimal(3000)
        elif token_name == 'DAI':
            return Decimal(1)
        else:
            raise Exception('price not found')
    
    def now(self):
        return int(datetime.datetime.now().timestamp())
    
    def refresh_prices(self, token_name):
        updated_at_timestamp, _ = self.token_prices[token_name]
        if self.is_price_too_old(updated_at_timestamp):
            self.add_fresh_token_price(token_name)
        
    def add_fresh_token_price(self, token_name):
        self.token_prices[token_name] = [self.now(),self.get_price(token_name)]
    
    def deposit(self, depositor, token_name, token_quantity):
        
        self.refresh_prices(token_name)
        print('token_prices', self.token_prices)
        
        # get total value of fund
        old_usd_balance = Decimal(0)
        for temp_token_name, temp_token_balance in self.token_balances.items():
            _, temp_token_price = self.token_prices[temp_token_name]
            print ('HERE token {} temp_token_price {}'.format(temp_token_name, temp_token_price))
            token_usd_value = temp_token_price * temp_token_balance
            old_usd_balance += token_usd_value
        # sum with token_quantity * price
        print ('old usd balance {}'.format(old_usd_balance))
        _, token_price = self.token_prices[token_name]
        new_deposit = token_quantity * token_price
        # determine pct of depositor
        new_usd_balance = (new_deposit + old_usd_balance)
        #pdb.set_trace()
        pct_depositor = new_deposit / new_usd_balance
        
        # updates
        self.shareholders_share[depositor] = pct_depositor
        self.token_balances[token_name] += token_quantity
        
        # determine pct of previous depositors
        for address, pct_total_holdings in self.shareholders_share.items():
            if not address == depositor:
                new_pct = (pct_total_holdings*old_usd_balance) / new_usd_balance
                print ('new_pct {} pct_total_holdings {}'.format(new_pct, pct_total_holdings))
                self.shareholders_share[address] = new_pct
        
        print ('shareholders {} old_usd_balance {} new_usd_balance {}'.format(self.shareholders_share, old_usd_balance,
                                                                             new_usd_balance))
        print ('token balances {}'.format(self.token_balances))

In [4]:
# test deposit
f = Fund()
initial_balance = f.token_balances['DAI']
unittest.TestCase().assertEqual(initial_balance, 0, 'oi')
deposit_alice = 100
f.deposit('alice', 'DAI', deposit_alice)
unittest.TestCase().assertEqual(f.token_balances['DAI'], deposit_alice, 'ops')
unittest.TestCase().assertEqual(f.shareholders_share['alice'], Decimal(1), 'ops')

token_prices {'WETH': [31532400.0, 0], 'DAI': [1649011725, Decimal('1')]}
HERE token WETH temp_token_price 0
HERE token DAI temp_token_price 1
old usd balance 0
shareholders {'alice': Decimal('1')} old_usd_balance 0 new_usd_balance 100
token balances {'WETH': 0, 'DAI': 100}


In [5]:
deposit_bob = 300
f.deposit('bob', 'DAI', deposit_bob)
unittest.TestCase().assertEqual(f.token_balances['DAI'], deposit_alice + deposit_bob, 'token balance does not match')
unittest.TestCase().assertEqual(f.shareholders_share['alice'], Decimal(0.25), 'token pct Alice does not match')
unittest.TestCase().assertEqual(f.shareholders_share['bob'], Decimal(0.75), 'token pct Alice does not match')

token_prices {'WETH': [31532400.0, 0], 'DAI': [1649011725, Decimal('1')]}
HERE token WETH temp_token_price 0
HERE token DAI temp_token_price 1
old usd balance 100
new_pct 0.25 pct_total_holdings 1
shareholders {'alice': Decimal('0.25'), 'bob': Decimal('0.75')} old_usd_balance 100 new_usd_balance 400
token balances {'WETH': 0, 'DAI': 400}


In [6]:
deposit_eve = 600
f.deposit('eve', 'DAI', deposit_eve)
unittest.TestCase().assertEqual(f.token_balances['DAI'], deposit_alice + deposit_bob + deposit_eve)
unittest.TestCase().assertEqual(f.shareholders_share['alice'], Decimal('0.1'))
unittest.TestCase().assertEqual(f.shareholders_share['bob'], Decimal('0.3'))
unittest.TestCase().assertEqual(f.shareholders_share['eve'], Decimal('0.6'))

token_prices {'WETH': [31532400.0, 0], 'DAI': [1649011725, Decimal('1')]}
HERE token WETH temp_token_price 0
HERE token DAI temp_token_price 1
old usd balance 400
new_pct 0.10 pct_total_holdings 0.25
new_pct 0.30 pct_total_holdings 0.75
shareholders {'alice': Decimal('0.10'), 'bob': Decimal('0.30'), 'eve': Decimal('0.6')} old_usd_balance 400 new_usd_balance 1000
token balances {'WETH': 0, 'DAI': 1000}


In [7]:
# Test WETH
fweth = Fund()
deposit_weth = 1
fweth.deposit('alice2','WETH',deposit_weth)
unittest.TestCase().assertEqual(fweth.token_balances['WETH'], deposit_weth)
unittest.TestCase().assertEqual(fweth.shareholders_share['alice2'], Decimal(1))
deposit_dai = 3000
fweth.deposit('bob2','DAI',deposit_dai)
#unittest.TestCase().assertEqual(fweth.token_balances['DAI'], deposit_dai)
#unittest.TestCase().assertEqual(fweth.shareholders_share['alice2'], Decimal('0.5'))
#unittest.TestCase().assertEqual(fweth.shareholders_share['bob2'], Decimal('0.5'))

token_prices {'WETH': [1649011727, Decimal('3000')], 'DAI': [31532400.0, 0]}
HERE token WETH temp_token_price 3000
HERE token DAI temp_token_price 0
old usd balance 0
shareholders {'alice2': Decimal('1')} old_usd_balance 0 new_usd_balance 3000
token balances {'WETH': 1, 'DAI': 0}
token_prices {'WETH': [1649011727, Decimal('3000')], 'DAI': [1649011727, Decimal('1')]}
HERE token WETH temp_token_price 3000
HERE token DAI temp_token_price 1
old usd balance 3000
new_pct 0.5 pct_total_holdings 1
shareholders {'alice2': Decimal('0.5'), 'bob2': Decimal('0.5')} old_usd_balance 3000 new_usd_balance 6000
token balances {'WETH': 1, 'DAI': 3000}
